# 1

In [1]:
#!/usr/bin/env python3
import random, datetime, sys, csv
from random import randrange, randint

date_start = datetime.date(2000, 1, 1)
date_period = 365 * 17

# Reads in emails.txt and movies.txt and creates 'nbr_movies' entries for each
# email.
# Returns the database, the emails and the movies in the following format:
# [ [ user, movie, date, grade ], ... ]
def create_db(nbr_movies):
    with open("emails.txt") as f:
        emails = f.read().split("\n");
    while "" in emails:
        emails.remove("")

    with open("movies.txt") as f:
        movies = f.read().split("\n");
    while "" in movies:
        movies.remove("")

    db = []

    for email in emails:
        movies_index = list(range(0, len(movies)))
        random.shuffle(movies_index)
        for i, f in enumerate(movies_index[0:nbr_movies]):
            dat = date_start + datetime.timedelta(randint(1, date_period))
            db.append(
                [email, movies[f], dat.strftime("%Y/%m/%d"), randint(1, 5)])

    return db, emails, movies

In [2]:
db, emails, movies = create_db(20)

### 1a

In [3]:
# Anonymize the given database, but still let the get_movies_with_rating
# function give the right answers.
def anonymize_1(db):
    anon = [['*', e[1], '*', e[3]] for e in db]
    return anon

In [4]:
# For a given anonymized-database and a rating, this function should return
# the films with the given rating.
def get_movies_with_rating(anon, rating):
    return list(set([e[1] for e in anon if e[3] == rating]))

### 1b

In [5]:
# A bit lesser anonymization than anonymize_1, but still no date. The returned
# database should have enough information to be used by get_top_rated. If you
# use a too simple hashing-function like sha-256, the result will be rejected.
def anonymize_2(db):
    def superencryption(msg):
        import base64
        key = "Never send a human to do a machine's job"
        if (len(key) < len(msg)):
            key = key + key[0:len(msg) - len(key)]
        amsg = map(ord, msg)
        akey = map(ord, key[0:len(msg)])
        y = ''.join([chr(v ^ i) for (i,v) in zip(akey, amsg)])
        return base64.b64encode(y.encode('utf-8'))
    return [[superencryption(e[0]), e[1], '*', e[3]] for e in db]

In [12]:
# get_top_rated searches for all users having rated a movie and searches their
# top-rated movie(s). It returns a list of all found movies, also doubles!
def get_top_rated_pd(anon, movie):
    import pandas as pd
    anon_df = pd.DataFrame(anon)
    res = []
    for e in anon_df.groupby(0):
        if movie in list(e[1][1]):
            top = e[1][3].where(lambda x: x==max(e[1][3])).notnull()
            list_u = list(e[1][1].loc[top])
            res += list_u
    return res

In [16]:
max([1,2,3])

3

In [17]:
# get_top_rated searches for all users having rated a movie and searches their
# top-rated movie(s). It returns a list of all found movies, also doubles!
def get_top_rated(anon, movie):
    res = []
    users = list(set([e[0] for e in anon]))
    for u in users:
        movies_ratings = [[e[1], e[3]] for e in anon if e[0] == u]
        condition = movie in [e[0] for e in movies_ratings]
        if condition:
            max_rating = max([e[1] for e in movies_ratings])
            max_movies = [e[0] for e in movies_ratings if e[1] == max_rating]
            res += max_movies
    return res

In [18]:
assert (get_top_rated(anon_db2, movies[0]) == get_top_rated(anon_db2, movies[0]))